In [1]:
import spacy
from spacy.tokens import Span
import re

In [12]:
text = "Carlos Souza was an Student, but Carlos Silva is a British TV Host. The name Carlos is quite commun"

In [16]:
pattern = r"Carlos [A-Z]\w+" #encontrar qualquer instancia de Paul neste caso uma letra maiscula ate a palavra real

In [17]:
matches = re.finditer(pattern, text) #encontrar o padrao neste texto
for match in matches:
    print(match)

<re.Match object; span=(0, 12), match='Carlos Souza'>
<re.Match object; span=(33, 45), match='Carlos Silva'>


In [18]:
# Mudei esta parte do codigo, adicionando filtro para evitar insercao de entidades duplicadas

# Inicializando o modelo em branco
nlp = spacy.blank("en")
text = "John and Jane went to the market. Mr. Smith was there too."

# Definindo o padrão regex para identificar nomes
pattern = r"\b(?:John|Jane|Mr. Smith)\b"

# Processando o texto com o modelo
doc = nlp(text)

original_ents = list(doc.ents)  # lista original de entidades
mwt_ents = []  # lista para armazenar as novas entidades encontradas

# Iterando sobre as correspondências do regex
for match in re.finditer(pattern, doc.text):
    start, end = match.span()  # posição inicial e final do texto
    span = doc.char_span(start, end)  # define o span
    if span is not None:
        # Evita a duplicação de entidades
        if not any(ent.start == span.start and ent.end == span.end for ent in original_ents):
            mwt_ents.append((span.start, span.end, span.text))  # adiciona à lista de entidades

# Adicionando as novas entidades à lista original
for ent in mwt_ents:
    start, end, name = ent  # começo, fim e nome
    per_ent = Span(doc, start, end, label="PERSON")  # cria o objeto Span com o rótulo "PERSON"
    original_ents.append(per_ent)

# Atualizando as entidades do doc
doc.ents = original_ents

# Exibindo as entidades encontradas
for ent in doc.ents:
    print(ent.text, ent.label_)


John PERSON
Jane PERSON
Mr. Smith PERSON


In [19]:
print(mwt_ents) #esta neste intervalo de palavras os respectivos nomes

[(0, 1, 'John'), (2, 3, 'Jane'), (8, 10, 'Mr. Smith')]


In [20]:
# adicionado filtro para evitar insercao de entidades duplicadas aqui tambem

from spacy.language import Language

@Language.component("paul_ner")
def paul_ner(doc):
    pattern = r"Paul [A-Z]\w+"  # Encontrar qualquer instância de "Paul" seguido de uma palavra com a primeira letra maiúscula
    original_ents = list(doc.ents)
    mwt_ents = []  # Lista que será preenchida pelas entidades encontradas
    for match in re.finditer(pattern, doc.text):  # Itera sobre as correspondências do regex
        start, end = match.span()  # Posição inicial e final do texto
        span = doc.char_span(start, end)  # Define o span
        if span is not None:
            # Evita a duplicação de entidades
            if not any(ent.start == span.start and ent.end == span.end for ent in original_ents):
                mwt_ents.append((span.start, span.end, span.text))  # Adiciona à lista de entidades
    for ent in mwt_ents:
        start, end, name = ent  # Começo, fim e nome
        per_ent = Span(doc, start, end, label="PERSON")  # Cria o objeto Span com o rótulo "PERSON"
        original_ents.append(per_ent)
    doc.ents = original_ents  # Atualiza as entidades do doc
    return doc


In [21]:
nlp2 = spacy.blank("en")
nlp2.add_pipe("paul_ner")

<function __main__.paul_ner(doc)>

In [22]:
doc2 = nlp2(text)
print(doc2)

John and Jane went to the market. Mr. Smith was there too.


In [26]:
# mudei aqui tbm adicionnado filtro para evitar sobreposicao de spans com a funcao filter_spans

from spacy.language import Language
from spacy.util import filter_spans  # importar a funcao filter_spans

# definir o componente personalizado
@Language.component("cinema_ner")
def cinema_ner(doc):
    pattern = r"Hollywood"  # padrao regex para encontrar "Hollywood"
    original_ents = list(doc.ents)  # entidades originais do doc
    mwt_ents = []  # lista para armazenar as novas entidades

    # iterar sobre as correspondencias do regex
    for match in re.finditer(pattern, doc.text):
        start, end = match.span()  # posicao inicial e final da correspondencia
        span = doc.char_span(start, end, label="CINEMA")  # criar um span com o rotulo "CINEMA"
        if span is not None:  # verificar se o span é valido
            mwt_ents.append(span)  # adicionar à lista de novas entidades

    # adicionar as novas entidades à lista de entidades originais
    original_ents.extend(mwt_ents)

    # filtrar spans sobrepostos e resolver conflitos
    filtered_ents = filter_spans(original_ents)

    # atualizar as entidades do doc
    doc.ents = filtered_ents
    return doc


In [27]:
nlp3 = spacy.load("en_core_web_sm")
nlp3.add_pipe("cinema_ner")

<function __main__.cinema_ner(doc)>

In [28]:
doc3 = nlp3(text)
for ent in doc3.ents:
    print(ent.text, ent.label_)

John PERSON
Jane PERSON
Smith PERSON
